In [1]:
from model import ModelConfig, LlamaModel
from train import TrainerConfig, DataLoader, Trainer

from transformers import AutoTokenizer

In [2]:
tokenizer_id = "HuggingFaceTB/SmolLM2-135M"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)
tokenizer.pad_token = tokenizer.eos_token

In [4]:
model_config = ModelConfig(
    vocab_size=tokenizer.vocab_size,
    d_model=576,
    d_head=64,
    d_mlp_proj=1536,
    n_layers=30,
    n_kv_heads=3,
    n_attn_heads=9,
    rms_norm_eps=1e-5,
    initializer_range=0.041666666666666664,
    rope_theta=100000.0,
    padding_idx=tokenizer.pad_token_id
)

In [5]:
train_config = TrainerConfig(
    per_device_train_batch_size=8,
    max_seq_len=2048,
    num_epochs=20,
    eval_interval_steps=100,
    learning_rate=2e-3,
    log_dir="runs/shakespeare"
)

In [6]:
with open("data/tiny_shakespeare.txt") as f:
    text = f.read()

In [7]:
model = LlamaModel(model_config)
dataloader = DataLoader(train_config, tokenizer, text=text)
trainer = Trainer(train_config, model)

Total tokens                   | 342,016
Num Trainable Params           | 162,826,560
Train device                   | cuda, NVIDIA GeForce RTX 3090, N=1
Training precision             | torch.bfloat16
Flash Attention                | True
torch.compile()                | True





In [8]:
trainer.train(dataloader)

Training steps                 | 380 
Step: 0, Training Loss: 11.26471, Tokens/sec: 1446.8442643655137
Computing Eval loss, steps: 2
Step: 0, Eval Loss: 9.97953
Step: 1, Training Loss: 10.04189, Tokens/sec: 1700.9048912118005
Step: 2, Training Loss: 13.90497, Tokens/sec: 87907.76110828476
Step: 3, Training Loss: 9.56660, Tokens/sec: 87940.89869378602
Step: 4, Training Loss: 9.34001, Tokens/sec: 86717.57252231214
Step: 5, Training Loss: 8.73359, Tokens/sec: 89101.00748959089
Step: 6, Training Loss: 8.24238, Tokens/sec: 88563.37954980384
Step: 7, Training Loss: 7.67133, Tokens/sec: 86404.15442009721
Step: 8, Training Loss: 7.83910, Tokens/sec: 84444.58094768302
Step: 9, Training Loss: 7.22471, Tokens/sec: 86252.38484029603
Step: 10, Training Loss: 6.94995, Tokens/sec: 84090.67894838427
Step: 11, Training Loss: 6.71106, Tokens/sec: 87275.43867014289
Step: 12, Training Loss: 6.71211, Tokens/sec: 84603.42958254786
Step: 13, Training Loss: 6.76546, Tokens/sec: 88622.89087750185
Step: 14, Tra

In [9]:
input_ids = tokenizer(["HAMLET:\nTo be or"], return_tensors="pt")['input_ids'].to(trainer.device)
idx = model.generate(input_ids, temperature=0.25, top_k=50, max_new_tokens=64)
print(tokenizer.batch_decode(idx)[0])

HAMLET:
To be or the
re I is I's
And I is aer, and the king,
And I have not not be the king,
And, I have not not a king,
And, I have been, and I will be,
And, and a man, and I will be
And,
